In [2]:
import pandas as pd
import mplfinance as mpf
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [3]:
# Create a DataFrame with the provided data
data = pd.read_csv("NSEBANK.csv")
column_name = ["Symbol", "Date", "Time", "Open", "High", "Low", "Close", "Volume", "OpenInterest"]
data.columns = column_name

In [4]:
data.head()

,Symbol,Date,Time,Open,High,Low,Close,Volume,OpenInterest
0,.NSEBANK,2022/01/03,09:16,35666.7,35681.9,35655.8,35658.8,0,0
1,.NSEBANK,2022/01/03,09:17,35658.1,35658.1,35618.1,35640.9,0,0
2,.NSEBANK,2022/01/03,09:18,35637.0,35685.7,35637.0,35684.1,0,0
3,.NSEBANK,2022/01/03,09:19,35682.0,35698.6,35672.6,35691.9,0,0
4,.NSEBANK,2022/01/03,09:20,35689.1,35696.9,35651.3,35651.3,0,0


In [5]:
data["DateTime"] = pd.to_datetime(data["Date"] + " " + data["Time"])
data.set_index("DateTime", inplace=True)
data.head()

,Symbol,Date,Time,Open,High,Low,Close,Volume,OpenInterest
DateTime,,,,,,,,,
2022-01-03 09:16:00,.NSEBANK,2022/01/03,09:16,35666.7,35681.9,35655.8,35658.8,0,0
2022-01-03 09:17:00,.NSEBANK,2022/01/03,09:17,35658.1,35658.1,35618.1,35640.9,0,0
2022-01-03 09:18:00,.NSEBANK,2022/01/03,09:18,35637.0,35685.7,35637.0,35684.1,0,0
2022-01-03 09:19:00,.NSEBANK,2022/01/03,09:19,35682.0,35698.6,35672.6,35691.9,0,0
2022-01-03 09:20:00,.NSEBANK,2022/01/03,09:20,35689.1,35696.9,35651.3,35651.3,0,0


candlestick = go.Candlestick(x=data.index, open=data['Open'],high=data['High'],
                             low=data['Low'], close=data['Close'])
fig = go.Figure(data=[candlestick])
fig.show()

In [130]:
data.loc["2022/01/03":"2022/01/10"]

,Symbol,Date,Time,Open,High,Low,Close,Volume,OpenInterest
DateTime,,,,,,,,,
2022-01-03 09:16:00,.NSEBANK,2022/01/03,09:16,35666.7,35681.9,35655.8,35658.8,0,0
2022-01-03 09:17:00,.NSEBANK,2022/01/03,09:17,35658.1,35658.1,35618.1,35640.9,0,0
2022-01-03 09:18:00,.NSEBANK,2022/01/03,09:18,35637.0,35685.7,35637.0,35684.1,0,0
2022-01-03 09:19:00,.NSEBANK,2022/01/03,09:19,35682.0,35698.6,35672.6,35691.9,0,0
2022-01-03 09:20:00,.NSEBANK,2022/01/03,09:20,35689.1,35696.9,35651.3,35651.3,0,0
...,...,...,...,...,...,...,...,...,...
2022-01-10 15:26:00,.NSEBANK,2022/01/10,15:26,38376.6,38382.6,38369.1,38377.1,0,0
2022-01-10 15:27:00,.NSEBANK,2022/01/10,15:27,38373.9,38400.4,38370.5,38374.6,0,0
2022-01-10 15:28:00,.NSEBANK,2022/01/10,15:28,38373.4,38382.4,38351.8,38370.9,0,0


In [6]:
import pandas as pd
from sklearn.cluster import KMeans

# Assuming your data is loaded into a variable called 'data'

# Step 1: Data Preprocessing
data['Date'] = pd.to_datetime(data['Date'])
daily_avg_prices = data.groupby('Date')[['High', 'Low']].mean()

# Step 2: Feature Selection
features = daily_avg_prices[['High', 'Low']]

# Step 3: Choosing K
chosen_k = 2

# Step 4: Applying K-means
kmeans = KMeans(n_clusters=chosen_k).fit(features)
daily_avg_prices['Cluster'] = kmeans.labels_

# Step 5: Identify Clusters
support_resistance_levels = daily_avg_prices.groupby('Cluster')[['High', 'Low']].mean()

# Step 6: Generating DataFrame
support_resistance_dfs = []

for cluster_label in support_resistance_levels.index:
    support_resistance_dfs.append(pd.DataFrame({
        'Date': daily_avg_prices[daily_avg_prices['Cluster'] == cluster_label].index,
        'Support': support_resistance_levels.loc[cluster_label, 'Low'],
        'Resistance': support_resistance_levels.loc[cluster_label, 'High']
    }))

support_resistance_df = pd.concat(support_resistance_dfs, ignore_index=True)

support_resistance_df.head()


,Date,Support,Resistance
0,2022-01-12,41080.246079,41101.748721
1,2022-02-02,41080.246079,41101.748721
2,2022-02-03,41080.246079,41101.748721
3,2022-02-04,41080.246079,41101.748721
4,2022-02-10,41080.246079,41101.748721


In [27]:
pd.merge(data,support_resistance_df,on=['Date']).drop(columns=['Symbol','Volume','OpenInterest'])

,Date,Time,Open,High,Low,Close,Support,Resistance
0,2022-01-03,09:16,35666.7,35681.9,35655.8,35658.8,35996.175676,36022.051907
1,2022-01-03,09:17,35658.1,35658.1,35618.1,35640.9,35996.175676,36022.051907
2,2022-01-03,09:18,35637.0,35685.7,35637.0,35684.1,35996.175676,36022.051907
3,2022-01-03,09:19,35682.0,35698.6,35672.6,35691.9,35996.175676,36022.051907
4,2022-01-03,09:20,35689.1,35696.9,35651.3,35651.3,35996.175676,36022.051907
...,...,...,...,...,...,...,...,...
92651,2022-12-30,15:26,43035.2,43080.4,43035.2,43077.0,41080.246079,41101.748721
92652,2022-12-30,15:27,43079.8,43094.9,43075.1,43075.3,41080.246079,41101.748721
92653,2022-12-30,15:28,43073.1,43080.9,43030.3,43034.1,41080.246079,41101.748721
92654,2022-12-30,15:29,43034.5,43049.0,43022.9,43038.5,41080.246079,41101.748721


In [26]:
df = pd.merge(data,support_resistance_df,on=['Date']).drop(columns=['Symbol','Volume','OpenInterest'])
# Make sure 'Date' and 'Time' are strings
df['Date'] = df['Date'].astype(str)
df['Time'] = df['Time'].astype(str)

# Convert Date and Time to datetime and set as index
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.set_index('Datetime', inplace=True)
df.drop(['Date', 'Time'], axis=1, inplace=True)

# Filter data for one month (e.g., January 2022)
start_date = '2022-01-01'
end_date = '2022-01-6'
df_one_month = df[start_date:end_date]

# Downsample to five-minute intervals
ohlc_dict = {
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Support': 'first',
    'Resistance': 'first'
}

df_resampled = df_one_month.resample('5T').apply(ohlc_dict)
df_resampled.dropna(inplace=True)
# Create the candlestick chart using Plotly
# Create the candlestick chart using Plotly
fig = go.Figure(data=[go.Candlestick(
    x=df_resampled.index,
    open=df_resampled['Open'],
    high=df_resampled['High'],
    low=df_resampled['Low'],
    close=df_resampled['Close']
)])

# Calculate the number of x-axis ticks to display
num_ticks = 10  # Adjust this number based on your preference

# Select a subset of tickvals and ticktext for better readability
tick_positions = list(range(0, len(df_resampled.index), len(df_resampled.index) // num_ticks))
tickvals = [df_resampled.index[i] for i in tick_positions]
ticktext = [date.strftime('%Y-%m-%d %H:%M') for date in tickvals]

# Customize the layout
fig.update_layout(
    title='Candlestick Chart with Support and Resistance',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True,
    
    # Set x-axis type to 'category'
    xaxis_type='category',
    
    # Set custom tickvals and ticktext for x-axis
    xaxis=dict(
        tickvals=tickvals,
        ticktext=ticktext
    )
)

# Show the interactive plot
fig.show()